In [125]:
import pandas as pd
from pathlib import Path

from fastai.gen_doc.nbdoc import *
from fastai.text import * 
from fastai import *
from fastai.docs import *

In [126]:
LYRIC_PATH = Path('../data/interim')

In [127]:
df = pd.read_csv('../data/interim/test_lyrics.csv')

In [128]:
df.head()

,msd_id,lyrics
0,TRAAAAW128F429D538,"\n\n[Verse 1]\nAlright, I might\nHave had a li..."
1,TRAAABD128F429CF47,"\n\nDarling, I don't know much\nBut I know I l..."
2,TRAAADZ128F9348C2E,\n\nSiento una pena muy honda dentro del alma\...
3,TRAAAEF128F4273421,\n\nAdam Ant/Marco Pirroni\nEvery girl is a so...
4,TRAAAFD128F92F423A,"\n\nI've just erased it's been a while, I've g..."


In [129]:
df[df.columns[0]] = len(df) * [0]
df.rename(columns={df.columns[0]:0, df.columns[1]:1}, inplace=True)
df.to_csv(LYRIC_PATH/'train.csv')
df.to_csv(LYRIC_PATH/'valid.csv')

In [130]:
data_lm = text_data_from_csv(Path(LYRIC_PATH), data_func=lm_data)

Tokenizing train.


Numericalizing train.
Tokenizing valid.


Numericalizing valid.


In [131]:
#download_wt103_model()

In [132]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'], drop_mult=0.5)
#learn.fit_one_cycle(10, 1e-2)

In [133]:
learn.unfreeze()

In [137]:
learn.fit(5)

ValueError: can't optimize a non-leaf Tensor

In [135]:
learn.fit_one_cycle(5, 1e-2)

ValueError: can't optimize a non-leaf Tensor

## Generate

In [117]:
from nltk import word_tokenize

In [122]:
m = learn.model

In [123]:
s = word_tokenize('/n test here')
t = LongTensor([data_lm.valid_ds.vocab.stoi[word] for word in s]).view(-1,1)

In [124]:
for i in range(50):
    res,*_ = m(t)
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    word = data_lm.valid_ds.vocab.itos[n.item()]
    print(word, end=' ')
    if word == '<eos>': break
    t = torch.cat((t, n.unsqueeze(0).unsqueeze(0)))

AttributeError: 'RNNCore' object has no attribute 'hidden'